In [3]:
!pip install git+git://github.com/GerbenBeintema/gym-unbalanced-disk@master

  Cloning git://github.com/GerbenBeintema/gym-unbalanced-disk (to revision master) to c:\users\s153325\appdata\local\temp\pip-req-build-_1ttwlh0
  Created wheel for gym-unbalanced-disk: filename=gym_unbalanced_disk-0.2.1-py3-none-any.whl size=125363 sha256=afc414711bcf2a79c39cf6b204dfb4eb9c4cbee71b337b37f7d5aa8c19456081
  Stored in directory: C:\Users\s153325\AppData\Local\Temp\pip-ephem-wheel-cache-jxu7lqfh\wheels\b5\23\30\720738515953d9041dc553c0a527be7dfdf452dd25e0c5cd03
Successfully built gym-unbalanced-disk


In [1]:
import gym, time
import gym_unbalanced_disk #is required
import numpy as np
env = gym.make('unbalanced-disk-v0',dt=0.05,umax=4) #if this fails restart the kernel or use gym_unbalanced_disk.UnbalancedDisk
env_sin = gym.make('unbalanced-disk-sincos-v0',dt=0.05,umax=4) # angle expressed in sine and cos components. (useful for RL)


# First test

Look at the rewards

In [16]:
Umax = 4

T = 20 
ulist = np.sign(np.sin(np.arange(200)/T*2*np.pi))*Umax 

obs = env.reset()
# obs_s = env_sin.reset()
max_reward = 0
min_reward = 0.0001
try:
    for u in ulist:
        obs, reward, done, info = env.step(u)
        # Check rewards
        if reward>max_reward: max_reward=reward
        if reward<min_reward: min_reward=reward
#         obs_s, reward_s, done, info = env_sin.step(u)
        print(obs,reward)
#         print(obs_s,reward_s)
        env.render()
        time.sleep(1/24)
        if done:
            obs = env.reset()
#             obs_s  = env_sin.reset()
finally: #this will always run
    env.close()
#     env_sin.close()

[-0.0002582638386939873, 0.00235733733948034] 0.999999953634048
[0.03196076892199329, 2.527405456200002] 0.9974147450358009
[0.12476771492919196, 4.70608346147689] 0.9627789780840071
[0.26331688436716977, 6.406131444897161] 0.8415340985429317
[0.43973005439298257, 7.531086644438683] 0.6197141737677884
[0.6324787833174592, 8.076316820631545] 0.36715922963951836
[0.8384575003094044, 8.098747649512886] 0.17457189289990005
[1.0375770214305478, 7.714121911026283] 0.06931892054106994
[1.2229410125197913, 7.050279819107475] 0.024566301932845994
[1.3873577111395023, 6.2454627658384085] 0.008356835484265869
[1.5337926245069797, 5.404083853612106] 0.002906232153681338
[1.5932881458177368, -0.5297853843303855] 0.001821897179463489
[1.5108084332921115, -6.098687287403614] 0.0034693408046788284
[1.2936949185518862, -11.28575892028765] 0.015833291441656534
[0.9482561868262139, -15.892679693113083] 0.10593039450320814
[0.5059449196786364, -19.507861602652675] 0.5298834224551059
[-0.010319986273903263

[0.9773516156294519, -10.088494890537595] 0.09415148546825029
[0.6680881819196068, -14.270818577393584] 0.32897916664842997
[0.27198445836916035, -17.34293035529653] 0.832953768131626
[-0.18398861948732845, -18.97784038044263] 0.9178337199370087
[-0.666101709567645, -19.13205072995757] 0.3335339175757586
[-1.1326603233043708, -18.15623324764219] 0.04132643019366112
[-1.505533039040967, -11.551871496119222] 0.003642283038260255
[-1.7117393503934584, -5.254572436965187] 0.0006853290581483372
[-1.7710205301746036, 0.6161997192684938] 0.0004181656266615287
[-1.6834377754096046, 6.1385453348092325] 0.0008674936574395338
[-1.4661119703413348, 11.360315040475703] 0.004826136905070813
[-1.1225205806397545, 16.14843159310597] 0.044245309621107454
[-0.6674143181586546, 20.08513170775622] 0.3330943894175188
[-0.1285745326439953, 22.590469221369585] 0.9598479767634365
[0.44746510295188535, 23.290082661298076] 0.6064943051874209
[1.0241176962853809, 22.40724839662355] 0.07455083062426131
[1.4977906

In [3]:
print(max_reward)
print(min_reward)

0.9999999771190927
3.0102662325827384e-11


# Q-learning

In [4]:
from gym import Wrapper
from gym import spaces
from collections import deque #a list with a fixed lenght
class histwrapper(Wrapper):
    def __init__(self, env, Nhist=7):
        super(histwrapper, self).__init__(env) #sets self.env
        
        shape = env.observation_space.shape #load the obs shape
        shapehist = (Nhist,) + shape #add (Nhist)
        h = env.observation_space.high*np.ones(shapehist) #the upperbounds
        l = env.observation_space.low*np.ones(shapehist) #the lowerbounds
        self.observation_space = spaces.Box(low=l.astype(np.float32),high=h.astype(np.float32)) #modify the observation_space
        self.hist = deque([],Nhist)
        self.Nhist = Nhist
        
    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        self.hist.append(observation)
        observation = np.array(self.hist)
        return observation, reward, done, info

    def reset(self):
        observation = self.env.reset()
        self.hist.extend([observation]*self.Nhist)
        observation = np.array(self.hist)
        return observation
env = gym_unbalanced_disk.UnbalancedDisk(dt=0.025, umax=4.)

env = histwrapper(env, Nhist=4)
# env.reset()
# print(env.step(0))
# print(env.step(0))### Discretizer

### Discretizer

Since the observations are continuous we have to discretize these observations

In [15]:
class Discretize_obs(gym.Wrapper):
    def __init__(self, env, nvec=10):
        super(Discretize_obs, self).__init__(env) #sets self.env
        if isinstance(nvec,int): #nvec in each dimention
            self.nvec = [nvec]*np.prod(env.observation_space.shape,dtype=int)
        else:
            self.nvec = nvec
        self.nvec = np.array(nvec) #(Nobs,) array
        
        self.observation_space = gym.spaces.MultiDiscrete(self.nvec) 
        self.olow, self.ohigh = env.observation_space.low, env.observation_space.high 
        self.olow[0]=-1e12
        self.ohigh[0]=1e12
    def discretize(self,observation): 
        return tuple(((observation - self.olow)/(self.ohigh - self.olow+1e-12)*self.nvec).astype(int)) 
        
    def step(self, action):
        observation, reward, done, info = self.env.step(action) 
        return self.discretize(observation), reward, done, info 

    def reset(self):
        return self.discretize(self.env.reset()) 

env = gym_unbalanced_disk.UnbalancedDisk(dt=0.025, umax=4.)
#env = Discretize_obs(env,nvec=10)
env.reset()
print(env.step(0)) #testing
print(env.step(0))
print(env.step(1))
print(env.step(1))
print(env.step(2))
print(env.step(2))
print(env.step(2))
print(env.step(8))

([-0.00011401222068380449, -0.004491168081065118], 0.9999971994661668, False, {})
([0.001339352019579405, -0.007441451304690238], 0.9999978559047029, False, {})
([0.0093567068588872, 0.6225447188520417], 0.9998093898291149, False, {})
([0.030526589358134754, 1.1663685115469131], 0.9975635191901615, False, {})
([0.0727256841370284, 2.221221659391047], 0.9864591003831216, False, {})
([0.14110892300163744, 3.0413248588788435], 0.9522104301023266, False, {})
([0.22401317440282117, 3.5887721343414647], 0.8829264943007636, False, {})
([0.3259923413141902, 4.468508347901461], 0.7688287769841754, False, {})


In [19]:
def argmax(a):
    #random argmax
    a = np.array(a)
    return np.random.choice(np.arange(len(a),dtype=int)[a==np.max(a)])

def roll_mean(ar,start=2000,N=50): #smoothing if needed
    s = 1-1/N
    k = start
    out = np.zeros(ar.shape)
    for i,a in enumerate(ar):
        k = s*k + (1-s)*a
        out[i] = k
    return out

def Qlearn(env, nsteps=5000, callbackfeq=100, alpha=0.2,eps=0.2, gamma=0.99):
    env.action_space.n = 3
    from collections import defaultdict
    Qmat = defaultdict(float) #any new argument set to zero
    env_time = env
    while not isinstance(env_time,gym.wrappers.time_limit.TimeLimit):
        env_time = env_time.env
    ep_lengths = []
    ep_lengths_steps = []
    
    obs = env.reset()
    print('goal reached time:')
    for z in range(nsteps):

        if np.random.uniform()<eps:
            action = env.action_space.sample()
        else:
            action = argmax([Qmat[obs,i] for i in range(env.action_space.n)])

        obs_new, reward, done, info = env.step(action)

        if done and not info.get('TimeLimit.truncated', False): #terminal state = done and not by timeout
            #saving results:
            print(env_time._elapsed_steps, end=' ')
            ep_lengths.append(env_time._elapsed_steps)
            ep_lengths_steps.append(z)
            
            #updating Qmat:
            A = reward - Qmat[obs,action] # adventage or TD
            Qmat[obs,action] += alpha*A
            obs = env.reset()
        else: #done by timeout or not done
            A = reward + gamma*max(Qmat[obs_new, - Qmat[obs,action]
            Qmat[obs,action] += alpha*A
            obs = obs_new
            
            if info.get('TimeLimit.truncated',False): #done by timeout
                #saving results:
                ep_lengths.append(env_time._elapsed_steps)
                ep_lengths_steps.append(z)
                print('out', end=' ')
                
                #reset:
                obs = env.reset()
    print()
    
    return Qmat, np.array(ep_lengths_steps), np.array(ep_lengths)



for nvec in [5,10,20]: 
    max_episode_steps = 1000
    env = gym_unbalanced_disk.UnbalancedDisk(dt=0.025, umax=4.) 
    env = gym.wrappers.time_limit.TimeLimit(env,max_episode_steps=max_episode_steps) 
    env = Discretize_obs(env,nvec=nvec) 

    print('nvec=',nvec)
    Qmat, ep_lengths_steps, ep_lengths = Qlearn(env, nsteps=400_000, callbackfeq=5000) 
    plt.plot(ep_lengths_steps,roll_mean(ep_lengths,start=max_episode_steps),label=str(nvec)) 
plt.legend()
plt.show() 


nvec= 5
goal reached time:


TypeError: unhashable type: 'numpy.ndarray'